In [3]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import numpy as np
import torchvision 
import pandas as pd

In [4]:
train_data=pd.read_csv(r'F:\study\ml\HonyiLee2021\Lhy_Machine_Learning-main\01 Introduction\dataset\covid.train.csv',)
test_data=pd.read_csv(r'F:\study\ml\HonyiLee2021\Lhy_Machine_Learning-main\01 Introduction\dataset\covid.test.csv')

In [7]:
train_data.iloc[:5,[0,1,2,3,-3,-2,-1]]

,id,AL,AK,AZ,worried_become_ill.2,worried_finances.2,tested_positive.2
0,0,1.0,0.0,0.0,53.991549,43.604229,20.704935
1,1,1.0,0.0,0.0,54.185521,42.665766,21.292911
2,2,1.0,0.0,0.0,53.637069,42.972417,21.166656
3,3,1.0,0.0,0.0,52.446223,42.907472,19.896607
4,4,1.0,0.0,0.0,52.560315,43.321985,20.178428


In [13]:
train_data.shape

(2700, 95)

In [14]:
all_features=pd.concat((train_data.iloc[:,1:-1],test_data.iloc[:,1:]))

In [15]:
all_features.shape

(3593, 93)

transformer to Norm(0,1)

In [22]:
all_features.iloc[:,40:]=all_features.iloc[:,40:].apply(
lambda x: (x-x.mean())/ (x.std()))

In [24]:
n_train=train_data.shape[0]
train_features=torch.Tensor(all_features.iloc[:n_train,:].values)
test_features=torch.Tensor(all_features.iloc[n_train:,:].values)
train_labels=torch.Tensor(train_data.tested_positive.values).view(-1,1)

define net and func

In [25]:
loss=nn.MSELoss()

In [27]:
def rmse(net,features,labels):
    with torch.no_grad():
        rmse=torch.sqrt(2*loss(net(features),labels).mean())
    return rmse.item()

In [26]:
def get_net(num_features):
    net=nn.Sequential(
        nn.Linear(num_features,256),
        nn.ReLU(),
        nn.Linear(256,1)
    )
    return net

In [45]:
def train(net,train_features,train_labels,test_features,test_labels,num_epochs,
         learning_rate,weight_decay,batch_size):
    train_ls,test_ls=[],[]
    dataset=torch.utils.data.TensorDataset(train_features,train_labels)
    train_iter=torch.utils.data.DataLoader(dataset,batch_size,shuffle=True)
    optimizer=torch.optim.Adam(net.parameters(),lr=learning_rate,
                               weight_decay=weight_decay)
    for e in range(num_epochs):
        for x,y in train_iter:
            l=loss(net(x),y)
            optimizer.zero_grad()
            l.backward()
            optimizer.step()
        train_ls.append(rmse(net,train_features,train_labels))
        if test_labels is not None:
            test_ls.append(rmse(net,test_features,test_labels))
    return train_ls,test_ls
    

In [47]:
def get_k_fold_data(k,i,x,y):
    assert k>1
    fold_size=x.shape[0]//k
    x_train,y_train=None,None
    for j in range(k):
        idx=slice(j*fold_size,(j+1)*fold_size)
        x_part,y_part=x[idx,:],y[idx]
        if j==i:
            x_valid,y_valid=x_part,y_part
        elif x_train is None:
            x_train,y_train=x_part,y_part
        else:
            x_train=torch.cat((x_train,x_part))
            y_train=torch.cat((y_train,y_part))
    return x_train,y_train,x_valid,y_valid

In [48]:
def k_fold(k,x_train,y_train,num_epochs,lr,weight_decay,batch_size):
    train_l_sum,valid_l_sum=0,0
    for i in range(k):
        data=get_k_fold_data(k,i,x_train,y_train)
        net=get_net(x_train.shape[1])
        train_ls,valid_ls=train(net,*data,num_epochs,lr,weight_decay,batch_size)
        train_l_sum+=train_ls[-1]
        valid_l_sum+=valid_ls[-1]
        print('fold %d ,train rmse %f,valid rmse %f ' % (
        i,train_ls[-1],valid_ls[-1]))
    return train_l_sum/k,valid_l_sum/k

In [51]:
k,num_epochs,lr,weight_decay,batch_size=10,100,0.01,0,64

In [52]:
train_l,valid_l=k_fold(k,train_features,train_labels,num_epochs,lr,
                      weight_decay,batch_size)
print('%d-fold validation : avg train rmse %f,avg valid rmse %f' % (
k,train_l,valid_l))

fold 0 ,train rmse 0.246721,valid rmse 5.879383 
fold 1 ,train rmse 0.229612,valid rmse 4.496234 
fold 2 ,train rmse 0.277827,valid rmse 5.432685 
fold 3 ,train rmse 0.900608,valid rmse 4.927233 
fold 4 ,train rmse 0.283372,valid rmse 4.312486 
fold 5 ,train rmse 0.258530,valid rmse 5.116784 
fold 6 ,train rmse 0.372682,valid rmse 3.691861 
fold 7 ,train rmse 0.300368,valid rmse 4.590098 
fold 8 ,train rmse 0.328326,valid rmse 4.868841 
fold 9 ,train rmse 0.344429,valid rmse 4.547595 
10-fold validation : avg train rmse 0.354247,avg valid rmse 4.786320
